# Course Project: Wildfire Analysis (Part 1: Common Analysis)

## Data Acquisition

More and more frequently summers in the western US have been characterized by wildfires with smoke billowing across multiple western states. There are many proposed causes for this: climate change, US Forestry policy, growing awareness, just to name a few. Regardless of the cause, the impact of wildland fires is widespread as wildfire smoke reduces the air quality of many cities. There is a growing body of work pointing to the negative impacts of smoke on health, tourism, property, and other aspects of society.

The Course Project is designed to analyze the impacts of wildfires on specific cities in the United States, particularly focusing on the effects of wildfire smoke and its implications for public health, air quality, and overall community well-being.

This is the first step in the project, where all the students conduct a base analysis using a shared dataset while focusing on a unique city. The aim is to create an understanding of wildfire impacts tailored to local contexts.

In this notebook, we primarily obtain the Access to the historical data of wildfires, and compute the average distance between the city assigned to me and the fire perimeter as this is required to obtain the smoke estimate. We will talk more about this estimate in the next notebook.

**License**:
Many of the snippets used here was developed by Dr. David W. McDonald for use in DATA 512, a course in the UW MS Data Science degree program. This code is provided under the [Creative Commons](https://creativecommons.org) [CC-BY license](https://creativecommons.org/licenses/by/4.0/). Revision 1.1 - August 16, 2024

## 1. Import libraries and required dependencies

In [1]:
# !pip install pyproj

In [2]:
# Import some standard python modules
import json

# The module pyproj is a standard module that can be installed using pip or your other favorite 
# installation tool. This module provides tools to convert between different geodesic coordinate systems
# and for calculating distances between points (coordinates) in a specific geodesic system.
from pyproj import Transformer, Geod

# The 'wildfire' module is a user module. This module is available from the course website. The module
# includes one object, a Reader, that can be used to read the GeoJSON files associated with the
# wildefire dataset. The module also contains a sample datafile that is GeoJSON compliant and that
# contains a small number of wildfires extracted from the main wildfire dataset.   
from wildfire.Reader import Reader as WFReader

# import pandas - we will have the valid attributes finally as a csv
import pandas as pd

# For better readability, I clear the cell output when running code in batches
from IPython.display import clear_output

I have been assigned to analyse the city: Vancouver, WA for my analysis
- 2023 estimate: 196442
- 2020 census: 190915
- 2020 density: 3920
- Latitude/Longitude: 45.64°N 122.60°W

In [3]:
#    CONSTANTS
#
#    The 'Wildfire_short_sample_2024.json' is an extraction from the full 'USGS_Wildland_Fire_Combined_Dataset.json'
#    dataset extracting several major wildfires in California, plus a couple others that have interested data structure
#    features. 
#
#    The sample file includes data for 15 fires, mostly oriented around the uniqueness of the name. Naming conventions
#    for wildfires is really adhoc, which makes finding any named fire in the dataset a disambiguation mess. The named
#    fires were selected from https://en.wikipedia.org/wiki/List_of_California_wildfires
#
#    The point nof the sample is to provide something small to test with before committing to processing the, much 
#    larger, full dataset.
#
EXTRACT_FILENAME = "../data/input_data/USGS_Wildland_Fire_Combined_Dataset.json"
DATA_FILENAME = EXTRACT_FILENAME
#
# print out where we think we're going to find the sample data
print(f"{DATA_FILENAME=}")

# A dictionary of city location from the US west coast states.
CITY_LOCATIONS = {
    'vancouver' :     {'city'   : 'Vancouver',
                       'latlon' : [45.64, -122.60] }
}

DATA_FILENAME='../data/input_data/USGS_Wildland_Fire_Combined_Dataset.json'


## 2. Data Acquisition

The primary dataset used in this project is the [Combined Wildland Fire Datasets for the United States and Certain Territories (1800s-Present)](https://www.sciencebase.gov/catalog/item/61aa537dd34eb622f699df81), which is sourced from the US Geological Survey (USGS). This dataset is particularly notable for several key characteristics:
- **Comprehensive:** It merges data from 40 different sources into a single dataset that provides detailed information on wildfires and prescribed fires across the United States from the mid-1800s to the present.
- **Geographic Coverage:** Includes entire United States and certain territories, making it applicable for a variety of urban analyses.
- **Variables Included:** The dataset includes essential attributes such as: Fire Name, Ignition Date, Controlled Date, Containment Date, Fire Cause, Fire Polygons
-  **Temporal Range:** The dataset covers a long historical period, allowing for analyses of trends over time and the assessment of changes in wildfire patterns and impacts.
- **Summary Statistics:** It provides spatial summary statistics, such as the frequency of fires in specific geographic areas, enabling a more robust analysis of smoke impacts on communities over the years.
- **Data Formats:** Available in both ArcGIS and GeoJSON formats, making it accessible for various data processing and geographic information system (GIS) applications.

The dataset is intended for public use, allowing researchers, students, and policymakers to access and analyze data related to wildfires.​ Users are encouraged to employ the data responsibly, ensuring that analyses and representations accurately reflect the information contained within the datasets.

### 2.1 Load the wildfire data using the wildfire Reader object
In the following cells we provide small code snippets that do the following:
- Create a wildfire Reader() object and use it to open the sample data file. Once, opened, we have access to the header information so we print that to show the structure of that data.
- Use the Reader() object and the next() method to read a set of wildfire features. The small sample file should have 15 of them. 
- Print one example feature showing the dictionary data structure of a feature.
- Access the geometry of one specific feature to get the 'ring' boundary of that specific fire - which is a list of geodetic coordinates.

Note that some of the output cells are quite long. Once you understand what they are illustrating you might want to collapse them or comment out the print statements that generate the output.

Another note regarding terminology. In the GeoJSON standard, something that is to be represented geographically is generically called a 'feature'. In the case of the wildfire dataset every 'feature' is a wildfire. These terms are used somewhat interchangably below.

In [4]:
#    This bit of code opens a new wildfire reader, gets the header information and prints it to the screen
print(f"Attempting to open '{DATA_FILENAME}' with wildfire.Reader() object")
wfreader = WFReader(DATA_FILENAME)
print()

#    Now print the header - it contains some useful information
header_dict = wfreader.header()
header_keys = list(header_dict.keys())
print("The header has the following keys:")
print(header_keys)
print()
print("Header Dictionary")
print(json.dumps(header_dict,indent=4))

Attempting to open '../data/input_data/USGS_Wildland_Fire_Combined_Dataset.json' with wildfire.Reader() object

The header has the following keys:
['displayFieldName', 'fieldAliases', 'geometryType', 'spatialReference', 'fields']

Header Dictionary
{
    "displayFieldName": "",
    "fieldAliases": {
        "OBJECTID": "OBJECTID",
        "USGS_Assigned_ID": "USGS Assigned ID",
        "Assigned_Fire_Type": "Assigned Fire Type",
        "Fire_Year": "Fire Year",
        "Fire_Polygon_Tier": "Fire Polygon Tier",
        "Fire_Attribute_Tiers": "Fire Attribute Tiers",
        "GIS_Acres": "GIS_Acres",
        "GIS_Hectares": "GIS_Hectares",
        "Source_Datasets": "Source Datasets",
        "Listed_Fire_Types": "Listed Fire Types",
        "Listed_Fire_Names": "Listed Fire Names",
        "Listed_Fire_Codes": "Listed Fire Codes",
        "Listed_Fire_IDs": "Listed Fire IDs",
        "Listed_Fire_IRWIN_IDs": "Listed Fire IRWIN IDs",
        "Listed_Fire_Dates": "Listed Fire Dates",
   

In [5]:
#    This code will load the whole sample file, or a small amount of the complete dataset.
#MAX_FEATURE_LOAD = 100
feature_list = list()
feature_count = 0

# A rewind() on the reader object makes sure we're at the start of the feature list
# This way, we can execute this cell multiple times and get the same result
wfreader.rewind()

# Now, read through each of the features, saving them as dictionaries into a list
feature = wfreader.next()
while feature:
    feature_list.append(feature)
    feature_count += 1
    # if we're loading a lot of features, print progress
    if (feature_count % 10000) == 0:
        print(f"Loaded {feature_count} features")
    # loaded the max we're allowed then break
    # if feature_count >= MAX_FEATURE_LOAD:
    #     break
    feature = wfreader.next()

#    Print the number of items (features) we think we loaded
print(f"Loaded a total of {feature_count} features")

#    Just a validation check - did all the items we loaded get into the list?
print(f"Variable 'feature_list' contains {len(feature_list)} features")

Loaded 10000 features
Loaded 20000 features
Loaded 30000 features
Loaded 40000 features
Loaded 50000 features
Loaded 60000 features
Loaded 70000 features
Loaded 80000 features
Loaded 90000 features
Loaded 100000 features
Loaded 110000 features
Loaded 120000 features
Loaded 130000 features
Loaded a total of 135061 features
Variable 'feature_list' contains 135061 features


The 'feature_list' variable was created when we read the sample file in a code cell above. To look at a single feature, uncomment and run the code. There are a lot of features, so I have commented it out to avoid space issues while uploading to github.

In [6]:
SLOT = 0
wf_feature = feature_list[SLOT]
# print(f"The wildfire feature from slot '{SLOT}' of the loaded 'feature_list'")
# print(json.dumps(wf_feature, indent=4))

Every feature has a 'geometry' which specifies geo coordinates that make up each geographic thing. In the case of the wildfire data, most wildfires are bounded shapes, circles, squares, etc. This is represented by shapes called 'rings' in GeoJSON.

In [7]:
# Get the geometry for the feature we pulled from the feature_list
wf_geometry = wf_feature['geometry']
# The largest shape (ring) is supposed to be item zero in the list of 'rings'
wf_bigest_ring = wf_geometry['rings'][0]

print(f"The largest ring of wf_feature['features'][{SLOT}]['rings'] consists of {len(wf_bigest_ring)} points.")

The largest ring of wf_feature['features'][0]['rings'] consists of 768 points.


GeoPandas offers geometrical types to represent spatial data, including both curve rings and standard rings.
- **Standard Rings:** Typically refer to polygonal geometries, which are defined as a sequence of connected line segments that entirely enclose a space. Each polygon may contain one or more rings:
    - Exterior Ring: The outer boundary of a polygon.
    - Interior Rings: Also known as holes, these are areas within the polygon that are not part of it. They represent exclusions within the polygon area.
- **Curve Rings:** Geometrical representations that utilize curved line segments. In GeoPandas, the use of curve geometries enables the modeling of more complex shapes through functions like offset_curve. These curves can create smoother transitions and are particularly beneficial in cases where the geometry reflects natural features or when modeling phenomena such as fire perimeters and smooth boundaries.

### 2.2 Convert points between geodetic coordinate systems

One of the constraints in doing geodetic computations is that most of the time we need to have our points (the coordinates for places) in the same geographic coordinate system. There are tons and tons of coordinate systems. You can find descriptions of many of them at [EPSG.io](https://epsg.io).

Looking at the wildfire header information, you can find this in the output of Example 1 (above), we can see fields named "geometryType" and "spatialReference". This looks like:

        "geometryType": "esriGeometryPolygon",
        "spatialReference": {
            "wkid": 102008,
            "latestWkid": 102008
        },

This indicates that the geometry of our wildfire data are generic polygons and that they are expressed in a coordinate system with the well-known ID (WKID) 102008. This coordinate system is also known as [ESRI:102008](https://epsg.io/102008)

If you look back, you might have wondered about the line of code that says:

    geocalc = Geod(ellps='WGS84')         # Use WGS84 ellipsoid representation of the earth

That string, 'WGS84', is a representation of the earth, that also relies on a well known coordinate system that is sometimes called 'decimal degrees' (DD). That decimal degrees system has an official name (or WKID) of [EPSG:4326](https://epsg.io/4326).

In the code below, we're going to do is take the geometry of a fire feature, extract the largest ring (i.e., the largest boundary of the fire) and convert all of the points in that ring from the ESRI:102008 coordinate system to EPSG:4326 coordinates. It turns out that the easy way to get locations for a city is to make a Google query for the location. That will respond with a decimal degress version of latitued and longitude. So, this allows us to move everything into decimal degress before we perform a distance calcluation.


In [8]:
#    Transform feature geometry data
#    The function takes one parameter, a list of ESRI:102008 coordinates that will be transformed to EPSG:4326
#    The function returns a list of coordinates in EPSG:4326
def convert_ring_to_epsg4326(ring_data=None):
    converted_ring = list()

    # We use a pyproj transformer that converts from ESRI:102008 to EPSG:4326 to transform the list of coordinates
    to_epsg4326 = Transformer.from_crs("ESRI:102008","EPSG:4326")
    # We'll run through the list transforming each ESRI:102008 x,y coordinate into a decimal degree lat,lon
    for coord in ring_data:
        lat,lon = to_epsg4326.transform(coord[0],coord[1])
        new_coord = lat,lon
        converted_ring.append(new_coord)
    return converted_ring

In [9]:
#   Convert one ring from the default to EPSG
ring_in_epsg4326 = convert_ring_to_epsg4326(wf_bigest_ring)
print(f"Ring consists of {len(ring_in_epsg4326)} points.")

#    If you want to print them out you can see what they look like converted - I have commented it out to avoid space issues.
# print(ring_in_epsg4326)
# for point in ring_in_epsg4326:
#    print(f"{point[0]},{point[1]}")

Ring consists of 768 points.


### 2.3 Compute distance between a place and a wildfire

The basic problem is knowing how far away a fire is from some location (like a city). One issue is that fires are irregularly shaped so the actual answer to that is a bit dependent upon the exact shape and how you want to think about the notion of 'distance'. For example, should we just find the closest point on the perimiter of a fire and call that the distance? Maybe we should find the centroid of the region, identify that as a geolocation (coordinate) and then calculate the distance to that? We can come up with numerous other ways.

The function defined below calculates the average distance of all perimeter points to the city (place) and returns that average as the distance. This is not quite what the centroid would be, but it is probably fairly close. This is one way to think about possible distance to a fire.

In [10]:
#    The function takes two parameters
#        A place - which is coordinate point (list or tuple with two items, (lat,lon) in decimal degrees EPSG:4326
#        Ring_data - a list of decimal degree coordinates for the fire boundary
#
#    The function returns the average miles from boundary to the place
def average_distance_from_place_to_fire_perimeter(place=None,ring_data=None):
    # convert the ring data to the right coordinate system
    ring = convert_ring_to_epsg4326(ring_data)    
    # create a epsg4326 compliant object - which is what the WGS84 ellipsoid is
    geodcalc = Geod(ellps='WGS84')
    # create a list to store our results
    distances_in_meters = list()
    # run through each point in the converted ring data
    for point in ring:
        # calculate the distance
        d = geodcalc.inv(place[1],place[0],point[1],point[0])
        distances_in_meters.append(d[2])
    # convert meters to miles
    distances_in_miles = [meters*0.00062137 for meters in distances_in_meters]
    # the esri polygon shape (the ring) requires that the first and last coordinates be identical to 'close the region
    # we remove one of them so that we don't bias our average by having two of the same point
    distances_in_miles_no_dup = distances_in_miles[1:]
    # now, average miles
    average = sum(distances_in_miles_no_dup)/len(distances_in_miles_no_dup)
    return average

According to the instructions, the smoke estimate should only consider fires within 650 miles of Vancouver, WA and that too for the past 60 years (1964 - 2024). We will handle this in the next notebook.

To compute the distance, we will make use of the above function that gets us the average distance to all the places in the perimeter. 

In [11]:
#  Get vancouver from our CITY_LOCATIONS constant as our starting position
place = CITY_LOCATIONS["vancouver"]

valid_attribute_list = []
rings_count = 0
curveRings_count = 0
error_count = 0

for idx, wf_feature in enumerate(feature_list):
    print(idx, "of", len(feature_list), "(", round(idx*100/len(feature_list), 1), "%)")
    wf_attributes = wf_feature['attributes']
    wf_id = wf_attributes['OBJECTID']
    wf_year = wf_attributes['Fire_Year']
    wf_name = wf_attributes['Listed_Fire_Names'].split(',')[0]
    wf_size =wf_attributes['GIS_Acres']

    # We will estimate the wildfire data for (1961-2024).
    if wf_year >= 1961:
        
        # Try to extract the first ring
        if 'rings' in wf_feature['geometry']:
            ring_data = wf_feature['geometry']['rings'][0]
            rings_count += 1
            #  Compute using the average distance to all points on the perimeter
            average_distance = average_distance_from_place_to_fire_perimeter(place['latlon'],ring_data)
            print(f"\nFire '{wf_name}' ({wf_size:1.2f} acres) from {wf_year} was an average {average_distance:1.2f} miles to {place['city']}")
        # Suppose there is no first ring, check if it has cureveRings
        elif 'curveRings' in wf_feature['geometry']:
            ring_data = wf_feature['geometry']['curveRings'][0]
            curveRings_count += 1
            average_distance = -1
            print("\n Skipped: Unable to calcualte the distance when the geometry is curved ring.")
        else:
            error_count += 1
            print("No compatible geometry in this fire data: ", wf_id)
            average_distance = -1

        # only consider valid fires within 650 miles from Vancouver
        if average_distance > -1:
                wf_attributes['average_distance'] = average_distance
                valid_attribute_list.append(wf_attributes)   
    else:
         print("\n Skipped: ", wf_year, " is not in the year range (1964 - 2024)")
    
    clear_output(wait=True)

# Create a DataFrame from the list of feature dictionaries
df = pd.DataFrame(valid_attribute_list)

135060 of 135061 ( 100.0 %)

Fire 'Oak Basin (1)' (0.97 acres) from 2020 was an average 92.76 miles to Vancouver


If the wildfire area can be represented as a polygon (for example, the area burned), we can use GeoPandas to create a Polygon representing the wildfire perimeter. The locations of interest (residential areas, facilities) can also be represented as points.

Standard Rings are typically defined as polygons where the perimeter is constructed from a series of straight-line segments. The current averge distance computation supports standard rings.

Curve Rings are more complex as they consist of curved line segments instead of straight edges. If we need to incorporate curve rings, we will have to handle them slightly differently since the coordinates will not form simple linear paths. This is not a part of the current implementation

In [12]:
print("Successfully obtained average distances for ", rings_count, "instances of standard rings.")
print("\nUnable to obtain distances for", curveRings_count, "instances as the geometrical type was Curved rings and") 
print("we cannot accurately measure the distance in these cases.")
# print(error_count, " had unknown geomertry. In these cases as well, we cannot accurately measure the distance.")

Successfully obtained average distances for  118465 instances of standard rings.

Unable to obtain distances for 35 instances as the geometrical type was Curved rings and
we cannot accurately measure the distance in these cases.


There were none with unknown geometry, so that's good!

I tried modifying the script to include complex geometries to handle the Curve Ring cases, however, existing libraries (such as shapely.geometry) were not built to handle complex cases. Since we only have 35 such instances, for now let us ignore these cases.

## 3 Save the acquired data

Let us now save the data for future reference.

In [13]:
df.to_csv("../data/intermediate_data/wildfire_dataset_with_distance.csv")
print("Successfully saved the acquired data!")

Successfully saved the acquired data!


We only have to obtain the wildfire data when the fire occured during the annual fire season (from May 1st through October 31st). We will take this condition into consideration while obtained the smoke estimate in the next step.
